## 5.10 批量归一化

> 1. **对于深层网络,即使输入数据做标准化,模型参数的更新依然会造成靠近输出层输出剧烈变化**
> 2. **批量归一化利用小批量上的均值和标准差,从而是输出更稳定**

### 5.10.1 批量归一化

#### 5.10.1.1 对全连接层做批量归一化

- 全连接层的输入: $\mu$
- 权重和偏差: $W$和$b$
- 激活函数: $\phi$
- 批量归一化运算符: $\mathbf{BF}$

批量归一化输入$x$由仿射变换 $$x = W\mu + b$$

由m个样本组成一个批次,射变换的输出为一个新的小批量$$\mathcal{B}=\{x(1),…,x(m)\}$$

则批量归一化输出层 $$y^{(i)} = \mathbf{BF}(x^{(i)})$$

1. 先对小批量$\mathcal{B}$求均值和方差:
    $$\mu_{\mathcal{B}} \leftarrow \frac{1}{m}\sum_{i=1}^{m} x^{(i)}$$
    $$\sigma_{\mathcal{B}}^2 \leftarrow \frac{1}{m}\sum_{i=1}^{m}(x^{(i)} - \mu_{\mathcal{B}})^2$$

2. 对$x^{(i)} \gt 0$做标准化,$\epsilon$ 为了防止分母大于0:
    $$\hat x^{(i)} \leftarrow  \frac{x^{(i)} - \mu_{\mathcal{B}}}{\sqrt {\sigma_\mathcal{B} + \epsilon}}$$
    
3. 批量归一化层引入两个参数,拉伸(scale)参数$\gamma$和偏移(shift)参数$\beta$,都和$x^{i}$形状一致,按元素乘法;所以$y^{(i)}$是$x^{(i)}$的批量归一化输出
    $$y^{(i)} \leftarrow \gamma \bigodot \hat x^{(i)} + \beta$$


4. 此时拉伸和偏移参数保留了不对$\hat x^{(i)}$做批量归一化的可能,可以理解为:如果批量归一化无益,理论上,学出的模型可以不使用批量归一化
    $$\gamma = \sqrt{\sigma_\mathcal{B} + \epsilon}$$
    $$\beta = \mu_{\mathcal{B}}$$

#### 5.10.1.2 对卷积层做批量归一化

1. 批量归一化发生在卷积计算之后、应用激活函数之前 
2. 如果卷积计算输出多个通道,需要分别做批量归一化,且每个通道都拥有独立的拉伸和偏移参数,并均为标量
3. 设小批量有m个元素,单个通道为$p \times q$,则需要对$m \times p \times q$个元素同时做批量性归一化

#### 5.10.1.3 预测时候批量归一化
1. 训练时,可将批量设的大一点,这样均值和方差较为准确;同时对任意输入有确定的输出
2. 所以单个样本的输出不应该取决于批量归一化的均值和方差
3. 常用方法: 通过移动平均估算整个训练数据集的样本均值和方差，并在预测时使用它们得到确定的输出

In [1]:
import time
import torch
from torch import nn, optim
import torch.nn.functional as F

import sys
sys.path.append("..") 
import d2lzh_pytorch.utils as d2l
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
def batch_norm(is_training, X, gamma, beta, moving_mean, moving_var, eps, momentum):
    # 判断当前是否是训练模式
    if not is_training:
        # 预测模式下,使用移动平均所得的均值和方差
        X_hat = (X - moving_mean) / torch.sqrt(moving_var + eps)
    else:
        assert len(X.shape) in (2, 4)
        if len(X.shape) == 2:
            # 使用全连接层的情况，计算特征维的均值和方差
            mean = X.mean(dim=0)
            var = ((X - mean) ** 2).mean(dim=0)
        else:
            # 使用二维卷积层的情况，计算通道维上（axis=1）的均值和方差。这里我们需要保持
            # X的形状以便后面可以做广播运算
            mean = X.mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
            var = ((X - mean) ** 2).mean(dim=0, keepdim=True).mean(dim=2, keepdim=True).mean(dim=3, keepdim=True)
        # 训练模式用当前的均值和方差
        X_hat = (X - mean) / torch.sqrt(var + eps)
        moving_mean = momentum * moving_mean + (1 - momentum) * mean
        moving_var = momentum * moving_var + (1 - momentum) * var
    Y = gamma * X_hat + beta
    return Y, moving_mean, moving_var

In [3]:
class BatchNorm(nn.Module):
    def __init__(self, num_features, num_dims):
        """
        @param: num_features: 对于全连接层为输出个数
        @param: num_dims: 对于全连接层和卷积层分别为2和4
        """
        super(BatchNorm, self).__init__()
        if num_dims == 2:
            shape = (1, num_features)
        else:
            shape = (1, num_features, 1, 1)
        # 参与求梯度和迭代的拉伸参数和偏移参数
        self.gamma = nn.Parameter(torch.ones(shape))
        self.beta = nn.Parameter(torch.zeros(shape))
        # 不参与梯度和迭代的变量
        self.moving_mean = torch.zeros(shape)
        self.moving_var = torch.zeros(shape)
        
    def forward(self, X):
        # 如果X不在内存上,则将moving_mean, moving_var 复制到显存上
        if self.moving_mean.device != X.device:
            self.moving_mean.to(X.device)
            self.moving_var.to(X.device)
        # 保存更新过后的moving_mean,moving_var. Module的实例training属性默认True,调用.eval()设置为False
        Y, self.moving_mean, self.moving_var = batch_norm(self.training, X,
                                                         self.gamma, self.beta, self.moving_mean,
                                                         self.moving_var, eps=1e-5, momentum=0.9)
        return Y

In [4]:
# 使用批量归一化层的LeNet
net = nn.Sequential(
    nn.Conv2d(1, 6, 5),  # in_channels, out_channels, kernel_size
    BatchNorm(6, num_dims=4), 
    nn.Sigmoid(),
    nn.MaxPool2d(2, 2),  # kernel_size, strid
    nn.Conv2d(6, 16, 5),
    BatchNorm(16, num_dims=4),
    nn.Sigmoid(),
    nn.MaxPool2d(2, 2),
    d2l.FlattenLayer(),
    nn.Linear(16*4*4, 120),
    BatchNorm(120, num_dims=2),
    nn.Sigmoid(),
    nn.Linear(120, 84),
    BatchNorm(84, num_dims=2),
    nn.Sigmoid(),
    nn.Linear(84, 10)
)

In [6]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size)

lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
# d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

### 5.10.3 简洁实现

> **`nn`模块中的`BatchNorm1d`和`BatchNorm2d`分别用于全连接层和卷积层,都需要指定输入的`num_features`**

In [7]:
net = nn.Sequential(
    nn.Conv2d(1, 6, 5),  # in_channels, out_channels, kernel_size
    nn.BatchNorm2d(6), 
    nn.Sigmoid(),
    nn.MaxPool2d(2, 2),  # kernel_size, strid
    nn.Conv2d(6, 16, 5),
    nn.BatchNorm2d(16),
    nn.Sigmoid(),
    nn.MaxPool2d(2, 2),
    d2l.FlattenLayer(),
    nn.Linear(16*4*4, 120),
    nn.BatchNorm1d(120),
    nn.Sigmoid(),
    nn.Linear(120, 84),
    nn.BatchNorm1d(84),
    nn.Sigmoid(),
    nn.Linear(84, 10)
)

In [8]:
batch_size = 256
train_iter, test_iter = d2l.load_data_fashion_mnist(batch_size=batch_size)

lr, num_epochs = 0.001, 5
optimizer = torch.optim.Adam(net.parameters(), lr=lr)
d2l.train_ch5(net, train_iter, test_iter, batch_size, optimizer, device, num_epochs)

training on  cpu
epoch 1, loss 0.9723, train acc 0.791, test acc 0.832, time 22.5 sec
epoch 2, loss 0.4512, train acc 0.866, test acc 0.849, time 22.4 sec
epoch 3, loss 0.3625, train acc 0.879, test acc 0.836, time 22.2 sec
epoch 4, loss 0.3270, train acc 0.888, test acc 0.855, time 22.6 sec
epoch 5, loss 0.3043, train acc 0.893, test acc 0.870, time 22.3 sec
